In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
from  text.phonemeic import phoneme_to_sequence
from text.new import use_phoneme

from scipy.io.wavfile import write
from convertor import cmu_to_ipa


def get_text(text, hps):
    # text_norm = text_to_sequence(text, hps.data.text_cleaners)
    # clean_text = phoneme_to_sequence(text, hps.data.text_cleaners)
    # print(text_norm)  # ->list
    text_norm = use_phoneme(text)
    print(text_norm)
    # print(clean_text)  # plˈiːz kˈɔːl kˈɔːl stˈɛlə  str
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    print(text_norm)
    return text_norm

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1042)
           2	LOAD_FAST(arg=0, lineno=1042)
           4	LOAD_CONST(arg=1, lineno=1042)
           6	BINARY_SUBSCR(arg=None, lineno=1042)
           8	LOAD_FAST(arg=0, lineno=1042)
          10	LOAD_CONST(arg=2, lineno=1042)
          12	BINARY_SUBSCR(arg=None, lineno=1042)
          14	COMPARE_OP(arg=4, lineno=1042)
          16	LOAD_FAST(arg=0, lineno=1042)
          18	LOAD_CONST(arg=1, lineno=1042)
          20	BINARY_SUBSCR(arg=None, lineno=1042)
          22	LOAD_FAST(arg=0, lineno=1042)
          24	LOAD_CONST(arg=3, lineno=1042)
          26	BINARY_SUBSCR(arg=None, lineno=1042)
          28	COMPARE_OP(arg=5, lineno=1042)
          30	BINARY_AND(arg=None, lineno=1042)
          32	RETURN_VALUE(arg=None, lineno=1042)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=104

In [1]:
from phonemizer import phonemize

text = "Please call Stella."
phonemes = phonemize(text, language='en-us', backend='espeak', strip=True, preserve_punctuation=True, with_stress=True)
print(phonemes)  # str

plˈiːz kˈɔːl stˈɛlə.


## LJ Speech

In [2]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./path/to/pretrained_ljs.pth", net_g, None)

In [4]:
stn_tst = get_text("p..p..p..p pl'i:z k'ɔ:l st st st'ɛlə", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1.3)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

[58, 4, 4, 58, 4, 4, 58, 4, 4, 58, 16, 58, 54, 176, 51, 2, 68, 16, 53, 176, 76, 2, 54, 16, 61, 62, 16, 61, 62, 16, 61, 62, 176, 86, 54, 83]
tensor([  0,  58,   0,   4,   0,   4,   0,  58,   0,   4,   0,   4,   0,  58,
          0,   4,   0,   4,   0,  58,   0,  16,   0,  58,   0,  54,   0, 176,
          0,  51,   0,   2,   0,  68,   0,  16,   0,  53,   0, 176,   0,  76,
          0,   2,   0,  54,   0,  16,   0,  61,   0,  62,   0,  16,   0,  61,
          0,  62,   0,  16,   0,  61,   0,  62,   0, 176,   0,  86,   0,  54,
          0,  83,   0])


## VCTK

In [6]:
hps = utils.get_hparams_from_file("./configs/vctk_base.json")

In [7]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./path/to/pretrained_vctk.pth", net_g, None)

In [9]:
stn_tst = get_text("p..p..p..plˈiːz kˈɔːl st..st..stˈɛlə.", hps)
print(stn_tst)   # tensor
# ipa_sequence = cmu_to_ipa("P sp P sp P L IY1 Z K AO1 L S sp S sp S T EH1 L AH0")
# print(ipa_sequence)
# P sp P sp P L IY1 Z K AO1 L S sp S sp S T EH1 L AH0
# AE1 S K HH ER0 T AH0 B R IH1 NG DH IY1 Z TH IH1 NG Z W IH0 DH HH ER0 F ER0 M DH AH0 S T AO1 R
# stn_tst = get_text(ipa_sequence, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([51]).cuda() # set speaker-id
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, sid=sid,noise_scale_w=0.8, length_scale=1.6)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

[58, 4, 4, 58, 4, 4, 58, 4, 4, 58, 54, 156, 51, 158, 68, 16, 53, 156, 76, 158, 54, 16, 61, 62, 4, 4, 61, 62, 4, 4, 61, 62, 156, 86, 54, 83, 4]
tensor([  0,  58,   0,   4,   0,   4,   0,  58,   0,   4,   0,   4,   0,  58,
          0,   4,   0,   4,   0,  58,   0,  54,   0, 156,   0,  51,   0, 158,
          0,  68,   0,  16,   0,  53,   0, 156,   0,  76,   0, 158,   0,  54,
          0,  16,   0,  61,   0,  62,   0,   4,   0,   4,   0,  61,   0,  62,
          0,   4,   0,   4,   0,  61,   0,  62,   0, 156,   0,  86,   0,  54,
          0,  83,   0,   4,   0])
tensor([  0,  58,   0,   4,   0,   4,   0,  58,   0,   4,   0,   4,   0,  58,
          0,   4,   0,   4,   0,  58,   0,  54,   0, 156,   0,  51,   0, 158,
          0,  68,   0,  16,   0,  53,   0, 156,   0,  76,   0, 158,   0,  54,
          0,  16,   0,  61,   0,  62,   0,   4,   0,   4,   0,  61,   0,  62,
          0,   4,   0,   4,   0,  61,   0,  62,   0, 156,   0,  86,   0,  54,
          0,  83,   0,   4,   0])


In [6]:
stn_tst = get_text("ˈæsk hɜː t..t..t..tə bɹˈɪŋ ðiːz θˈɪŋz wɪð hɜː fɹʌmðə st..st..st..stˈoːɹ", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([51]).cuda() # speaker id
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, sid=sid,noise_scale_w=0.8, length_scale=1.6)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

[156, 72, 61, 53, 16, 50, 87, 158, 16, 62, 4, 4, 62, 4, 4, 62, 4, 4, 62, 83, 16, 44, 123, 156, 102, 112, 16, 81, 51, 158, 68, 16, 119, 156, 102, 112, 68, 16, 65, 102, 81, 16, 50, 87, 158, 16, 48, 123, 138, 55, 81, 83, 16, 61, 62, 4, 4, 61, 62, 4, 4, 61, 62, 4, 4, 61, 62, 156, 57, 158, 123]
tensor([  0, 156,   0,  72,   0,  61,   0,  53,   0,  16,   0,  50,   0,  87,
          0, 158,   0,  16,   0,  62,   0,   4,   0,   4,   0,  62,   0,   4,
          0,   4,   0,  62,   0,   4,   0,   4,   0,  62,   0,  83,   0,  16,
          0,  44,   0, 123,   0, 156,   0, 102,   0, 112,   0,  16,   0,  81,
          0,  51,   0, 158,   0,  68,   0,  16,   0, 119,   0, 156,   0, 102,
          0, 112,   0,  68,   0,  16,   0,  65,   0, 102,   0,  81,   0,  16,
          0,  50,   0,  87,   0, 158,   0,  16,   0,  48,   0, 123,   0, 138,
          0,  55,   0,  81,   0,  83,   0,  16,   0,  61,   0,  62,   0,   4,
          0,   4,   0,  61,   0,  62,   0,   4,   0,   4,   0,  61,   0,  62,
       

### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))